In [ ]:
#@title B-SOiD GOOGLE COLAB 
# Created by Yttri Lab at Carnegie Mellon University.
# Program developer: Alexander Hsu.
# Program collaborator: Vishal Patel.
# Date last modified: 030420
# Contact: ahsu2@andrew.cmu.edu

# Import necessary python packages
import numpy as np
import math
import pandas as pd
import time
import os
# import glob
!pip install MulticoreTSNE
from MulticoreTSNE import MulticoreTSNE as TSNE
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn import mixture, svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# Defining a few functions. 

In [ ]:
#@title BOXCAR_CENTER
def boxcar_center(a, n):

  a1 = pd.Series(a)
  moving_avg = np.array(a1.rolling(window = n,min_periods=1).mean(center=True))
  
  return moving_avg 

In [ ]:
#@title ADP_FILT
def adp_filt(currDf=None,*args,**kwargs):

    lIndex = []
    xIndex = []
    yIndex = []
    currDf = np.array(currDf[1:])
    for header in range(len(currDf[0])):
        if currDf[0][header] == "likelihood":
            lIndex.append(header)
        elif currDf[0][header] == "x":
            xIndex.append(header)
        elif currDf[0][header] == "y":
            yIndex.append(header)
    print('Replacing data-driven low likelihood positions with the most recent highly probable position... \n')
    start_time = time.time()
    currDf = np.array(currDf)
    currDf1 = currDf[:,1:]
    datax=currDf1[:,np.array(xIndex)-1]
    datay=currDf1[:,np.array(yIndex)-1]
    data_lh=currDf1[:,np.array(lIndex)-1]
    currDf_filt = np.zeros((datax.shape[0]-1,(datax.shape[1])*2))
    perc_rect = []
    for i in range(data_lh.shape[1]):
        perc_rect.append(0)
    for x in range(data_lh.shape[1]):
        if x <= 2:
            a,b = np.histogram(data_lh[1:,x].astype(np.float))
            rise_a = np.where(np.diff(a) >= 0)
            if rise_a[0][0] > 1:
                llh = (b[rise_a[0][0]]+b[rise_a[0][0]-1])/2
            else:
                llh = (b[rise_a[0][1]]+b[rise_a[0][1]-1])/2
        else:
            llh = 0.2
        data_lh_float = data_lh[1:,x].astype(np.float)
        perc_rect[x] = len(np.where(data_lh_float < llh)) / data_lh.shape[1]
        currDf_filt[0,(2*x):(2*x + 2)] = np.hstack([datax[1,x],datay[1,x]])
        # replacing with most recent highly probable positions
        for i in range(1,data_lh.shape[0]-1):
            if data_lh_float[i] < llh:
                currDf_filt[i,(2*x):(2*x + 2)] = currDf_filt[i - 1,(2*x):(2*x + 2)]
            else:
                currDf_filt[i,(2*x):(2*x + 2)] = np.hstack([datax[i,x],datay[i,x]])
    currDf_filt = np.array(currDf_filt[1:])
    currDf_filt = currDf_filt.astype(np.float)
    print("--- High-pass filter took %s seconds ---" % (time.time() - start_time))
    
    return currDf_filt, perc_rect

In [ ]:
# Here I would like to save the data that I have, since it took a long time to process
# %   DATA    6-body parts (x,y) matrix outlining the tetrapod animal over time videotaped from the bottom looking up. Rows represents time.
# %           Columns 1 & 2 tracks snout; columns 3 to 6 tracks the two front paws; columns 7 to 10 tracks the two hind paws;
# %           columns 11 & 12 tracks the base of the tail.
# data[m][:,0:2]: "snout" --> left forepaw
# data[m][:,2:4]: "left forepaw" --> right forepaw
# data[m][:,4:6]: "right forepaw" --> nose
# data[m][:,6:]: "hindpaw" --> sugar pellet

fps = 100
win_len = np.int(np.round(0.05/(1/fps))*2-1)
print(win_len)
time.time()

# %%

# @title B-SOiD_ASSIGN { vertical-output: true, display-mode: "form" }

def bsoid_assign(clean_dlc_output, fps, comp, kclass, it, *args, **kwargs):
    win_len = np.int(np.round(0.05 / (1 / fps)) * 2 - 1)
    print('Obtaining features from dataset... \n')
    start_time = time.time()
    feats = list()
    for clean_trial in range(len(clean_dlc_output)):
        ## Obtain features, 4 distance features and 3 time-varying speed/angle features
        ## I WILL HAVE TO CHANGE THIS PART, LET'S SEE WHAT I CAN DO
        
        number_trials = len(clean_dlc_output[clean_trial])  # Number of trials imported for bsoid to run on
        
        # clean_dlc_output[clean_trial][:,0] = x-values snout
        # clean_dlc_output[clean_trial][:,1] = y-values snout
        # clean_dlc_output[clean_trial][:,2] = x-values fore paw 1
        # clean_dlc_output[clean_trial][:,3] = y-values fore paw 1
        # clean_dlc_output[clean_trial][:,4] = x-values fore paw 2
        # clean_dlc_output[clean_trial][:,5] = y-values fore paw 2
        # clean_dlc_output[clean_trial][:,6] = x-values hind paw 1
        # clean_dlc_output[clean_trial][:,7] = y-values hind paw 1
        # clean_dlc_output[clean_trial][:,8] = x-values hind paw 2
        # clean_dlc_output[clean_trial][:,9] = y-values hind paw 2
        # clean_dlc_output[clean_trial][:,10] = x-values base of tail
        # clean_dlc_output[clean_trial][:,11] = y-values base of tail
        
        # clean_dlc_output[clean_trial][:,0] = x-values left paw (forepaw 1)
        # clean_dlc_output[clean_trial][:,1] = y-values left paw (forepaw 1)
        # clean_dlc_output[clean_trial][:,2] = x-values right paw (forepaw 2)
        # clean_dlc_output[clean_trial][:,3] = y-values right paw (forepaw 2)
        # clean_dlc_output[clean_trial][:,4] = x-values snout
        # clean_dlc_output[clean_trial][:,5] = y-values snout
        # clean_dlc_output[clean_trial][:,6] = x-values pellet
        # clean_dlc_output[clean_trial][:,7] = y-values pellet
        
        forepaw_distance = clean_dlc_output[clean_trial][:, 0:2] - clean_dlc_output[clean_trial][:, 2:4]

        center_between_forepaws = np.vstack(((clean_dlc_output[clean_trial][:, 0] + clean_dlc_output[clean_trial][:, 2]) / 2, (clean_dlc_output[clean_trial][:, 1] + clean_dlc_output[clean_trial][:, 3]) / 2)).T
        length_center_between_forepaws = len(center_between_forepaws)
        
        point_center_between_forepaws = np.vstack(([center_between_forepaws[:, 0] - clean_dlc_output[clean_trial][:, 10], center_between_forepaws[:, 1] - clean_dlc_output[clean_trial][:, 11]])).T
        
        center_between_hindpaws = np.vstack((((clean_dlc_output[clean_trial][:, 6] + clean_dlc_output[clean_trial][:, 8]) / 2), ((clean_dlc_output[clean_trial][:, 7] + clean_dlc_output[clean_trial][:, 9]) / 2))).T
        point_center_between_hindpaws = np.vstack(([center_between_hindpaws[:, 0] - clean_dlc_output[clean_trial][:, 10], center_between_hindpaws[:, 1] - clean_dlc_output[clean_trial][:, 11]])).T
        
        point_snout = np.vstack(([clean_dlc_output[clean_trial][:, 0] - clean_dlc_output[clean_trial][:, 10], clean_dlc_output[clean_trial][:, 1] - clean_dlc_output[clean_trial][:, 11]])).T

        norm_forepaw_distance = np.zeros(number_trials)
        norm_point_center_between_forepaws = np.zeros(number_trials)
        norm_point_center_between_hindpaws = np.zeros(number_trials)
        norm_point_snout = np.zeros(number_trials)
        
        for trial in range(1, number_trials):
            # Distance is calculated by taking the norm (finding the magnitude) of the difference between the two forepaws' (x,y) values
            norm_forepaw_distance[trial] = np.array(np.linalg.norm(clean_dlc_output[clean_trial][trial, 2:4] - clean_dlc_output[clean_trial][trial, 4:6]))
            norm_point_center_between_forepaws[trial] = np.linalg.norm(point_center_between_forepaws[trial, :])
            norm_point_center_between_hindpaws[trial] = np.linalg.norm(point_center_between_hindpaws[trial, :])
            norm_point_snout[trial] = np.linalg.norm(point_snout[trial, :])
        
        ### At this point, the 4 distance metrics have been calculated 
        
        # smooth_norm_forepaw_distance = np.zeros(number_trials)
        # sn_cfp_norm_smth = np.zeros(number_trials)
        # sn_chp_norm_smth = np.zeros(number_trials)
        # sn_pt_norm_smth = np.zeros(number_trials)
        
        smooth_norm_forepaw_distance = boxcar_center(norm_forepaw_distance, win_len)
        
        smooth_norm_distance_snout_center_forepaws = boxcar_center(norm_point_snout - norm_point_center_between_forepaws, win_len)
        smooth_norm_distance_snout_center_hindpaws = boxcar_center(norm_point_snout - norm_point_center_between_hindpaws, win_len)
        smooth_norm_point_snout = boxcar_center(norm_point_snout, win_len)
        
        angle_point_snout = np.zeros(number_trials - 1)
        displacement_snout = np.zeros(number_trials - 1)
        smooth_angle_point_snout = np.zeros(number_trials - 1)
        smooth_displacement_snout = np.zeros(number_trials - 1)
        
        for trial in range(0, number_trials - 1):
            next_point_snout_3d = np.hstack([point_snout[trial + 1, :], 0])
            this_point_snout_3d = np.hstack([point_snout[trial, :], 0])
            cross_product_points = np.cross(next_point_snout_3d, this_point_snout_3d)
            angle_point_snout[trial] = np.dot(np.dot(np.sign(cross_product_points[2]), 180) / np.pi,
                                  math.atan2(np.linalg.norm(cross_product_points), np.dot(point_snout[trial, :], point_snout[trial + 1, :])))
            displacement_snout[trial] = np.linalg.norm(clean_dlc_output[clean_trial][trial + 1, 0:2] - clean_dlc_output[clean_trial][trial, 0:2])
        
        smooth_angle_point_snout = boxcar_center(angle_point_snout, win_len)
        smooth_displacement_snout = boxcar_center(displacement_snout, win_len)
        
        feats.append(np.vstack((smooth_norm_distance_snout_center_forepaws[1:], smooth_norm_distance_snout_center_hindpaws[1:], smooth_norm_forepaw_distance[1:],
                                smooth_norm_point_snout[1:], smooth_angle_point_snout[:], smooth_displacement_snout[:])))
    
    print("--- Feature extraction took %s seconds ---" % (time.time() - start_time))
    
    # Feature compilation
    start_time = time.time()
    
    if comp == 0:
        f_10fps = list()
        tsne_feats = list()
        labels = list()
        
    for feature_num in range(0, len(feats)):
        
        feats1 = np.zeros(len(clean_dlc_output[feature_num]))
        
        for window_index in range(round(fps / 10) - 1, len(feats[feature_num][0]), round(fps / 10)):
            
            if window_index > round(fps / 10) - 1:
                
                feats1 = np.concatenate((feats1.reshape(feats1.shape[0], feats1.shape[1]),
                                         np.hstack((np.mean((feats[feature_num][0:4, range(window_index - round(fps / 10), window_index)]), axis=1),
                                                    np.sum((feats[feature_num][4:7, range(window_index - round(fps / 10), window_index)]),
                                                           axis=1))).reshape(len(feats[0]), 1)), axis=1)
            else:
                feats1 = np.hstack((np.mean((feats[feature_num][0:4, range(window_index - round(fps / 10), window_index)]), axis=1),
                                    np.sum((feats[feature_num][4:7, range(window_index - round(fps / 10), window_index)]), axis=1))).reshape(
                    len(feats[0]), 1)
        
        print("--- Feature compilation took %s seconds ---" % (time.time() - start_time))
        
        if comp == 1:
            if feature_num > 0:
                f_10fps = np.concatenate((f_10fps, feats1), axis=1)
            else:
                f_10fps = feats1
        else:
            f_10fps.append(feats1)
            if len(f_10fps[feature_num]) < 15000:
                p = 50
                exag = 12
                lr = 200
            else:
                p = round(f_10fps[feature_num].shape[1] / 300)
                exag = round(f_10fps[feature_num].shape[1] / 1200)
                lr = round(np.log(f_10fps[feature_num].shape[1]) / 0.04)
            start_time = time.time()
            ## Run t-SNE dimensionality reduction
            np.random.seed(0)  # For reproducibility
            print(
                'Running the compiled data through t-SNE collapsing the 7 features onto 3 action space coordinates... \n')
            tsne_feats_i = TSNE(n_components=3, perplexity=p, early_exaggeration=exag, learning_rate=lr,
                                n_jobs=8).fit_transform(f_10fps[feature_num].T)
            tsne_feats.append(tsne_feats_i)
            print("--- TSNE embedding took %s seconds ---" % (time.time() - start_time))

            ## Run a Gaussian Mixture Model Expectation Maximization to group the t-SNE clusters
            start_time = time.time()
            gmm = mixture.GaussianMixture(n_components=kclass, covariance_type='full', tol=0.001, reg_covar=1e-06,
                                          max_iter=100, n_init=it, init_params='random').fit(tsne_feats_i)
            grp = gmm.predict(tsne_feats_i)
            labels.append(grp)
            print("--- Gaussian mixtures took %s seconds ---" % (time.time() - start_time))
            print(" Plotting t-SNE with GMM assignments... ")
            uk = list(np.unique(labels))
            uniqueLabels = []
            for i in labels:
                indexVal = uk.index(i)
                uniqueLabels.append(indexVal)
            R = np.linspace(0, 1, len(uk))
            color = plt.cm.hsv(R)
            fig = go.Figure(
                data=[go.Scatter3d(x=tsne_feats_i[:, 0], y=tsne_feats_i[:, 1], z=tsne_feats_i[:, 2], mode='markers',
                                   marker=dict(size=2.5, color=color[uniqueLabels], opacity=0.8))])
            fig.show()
            print('TADA! \n')
    if comp == 1:
        if len(f_10fps) < 15000:
            p = 50
            exag = 12
            lr = 200
        else:
            p = round(f_10fps.shape[1] / 300)
            exag = round(f_10fps.shape[1] / 1200)
            lr = round(np.log(f_10fps.shape[1]) / 0.04)
        start_time = time.time()
        ## Run t-SNE dimensionality reduction
        np.random.seed(0)  # For reproducibility
        print('Running the compiled data through t-SNE collapsing the 7 features onto 3 action space coordinates... \n')
        tsne_feats = TSNE(n_components=3, perplexity=p, early_exaggeration=exag, learning_rate=lr,
                          n_jobs=8).fit_transform(f_10fps.T)
        print("--- TSNE embedding took %s seconds ---" % (time.time() - start_time))
        ## Run a Gaussian Mixture Model Expectation Maximization to group the t-SNE clusters
        start_time = time.time()
        gmm = mixture.GaussianMixture(n_components=kclass, covariance_type='full', tol=0.001, reg_covar=1e-06,
                                      max_iter=100, n_init=it, init_params='random').fit(tsne_feats)
        labels = gmm.predict(tsne_feats)
        print("--- Gaussian mixtures took %s seconds ---" % (time.time() - start_time))
        print(" Plotting t-SNE with GMM assignments... ")
        uk = list(np.unique(labels))
        uniqueLabels = []
        for i in labels:
            indexVal = uk.index(i)
            uniqueLabels.append(indexVal)
        R = np.linspace(0, 1, len(uk))
        color = plt.cm.hsv(R)
        fig = go.Figure(data=[go.Scatter3d(x=tsne_feats[:, 0], y=tsne_feats[:, 1], z=tsne_feats[:, 2], mode='markers',
                                           marker=dict(size=2.5, color=color[uniqueLabels], opacity=0.8))])
        fig.show()
        print('TADA! \n')

    return f_10fps, tsne_feats, np.array(uniqueLabels), fig

In [ ]:
#@title B-SOiD_MDL2
def bsoid_mdl2(f_10fps=None,labels=None,hldout=None,cv_it=None,*args,**kwargs):   
    
    print('Parsing features into train and test sets')
    feats_T=f_10fps.T
    labels_T=labels.T
    np.random.seed(0)
    feats_train, feats_test, labels_train, labels_test = train_test_split(feats_T, labels_T, test_size=hldout, 
                                                                          random_state=0)
    start_time = time.time()
    print('Training an SVM classifier (kernel trick: Gaussian)... \n')
    clf = svm.SVC(gamma=0.005, C= 10, random_state=0)
    clf.fit(feats_train, labels_train)
    scores = cross_val_score(clf, feats_test, labels_test, cv=cv_it)
    print("--- Training classifier and performing cross-validation {} times took %s seconds ---".format(cv_it) % (time.time() - start_time))
    fig = plt.figure(num=None, figsize=(1.5, 2), dpi=300, facecolor='w', edgecolor='k')
    fig.suptitle("Performance on {} % Data".format(hldout*100), fontsize=8, fontweight='bold')
    ax = fig.add_subplot(111)
    ax.boxplot(scores, notch=None)
    ax.set_xlabel('SVM',fontsize=8)
    ax.set_ylabel('Accuracy',fontsize=8)
    
    return clf,fig,scores

# Load the pose estimate data (.csv) generated from [DeepLabCut](https://github.com/AlexEMG/DeepLabCut).

In [ ]:
#dir_mainAnimal = input('Provide the directory containing all animals and DeepLabCut data: ')
dir_mainAnimal = '/Volumes/SharedX/Neuro-Leventhal/analysis/mouseSkilledReaching/DLC_outDir/'

In [ ]:
# Load my data
all_files = [file for file in os.listdir(dir_mainAnimal) if file.endswith('.csv')]

# # ORIGINAL CONTENT FROM BSOID MASTER
# # Load your data
# # Step 1: change path to "/content/drive/My Drive/DeepLabCut/"
# # Step 2: change datelist to ["experiment1","experiment2","more"]
# path = "/content/drive/My Drive/Colab Notebooks/"
# datelist = ["041919","042219"] # get the folder name
# all_files = list()
# print('Compiling all files...')
# for dates in datelist:
#     for file in glob.glob(path + dates + "/*.csv"):
#         all_files.append(file)

li = []
li_filt = []
perc_rect_li = []
investigateFiles = []
print('High-pass filter for low-likelihood pose estimation:')
for filename in all_files:
    currDf = pd.read_csv(os.path.join(dir_mainAnimal,filename),low_memory=False)
    try:
        currDf_filt,perc_rect = adp_filt(currDf)
        li.append(currDf)
        perc_rect_li.append(perc_rect)
        li_filt.append(currDf_filt)
    except IndexError:
        investigateFiles.append(filename)
        continue
        
data = np.array(li_filt)
print('You have compiled .csv files into a',data.shape,'data list.')
data = data

# Feature extraction, dimensionality reduction (t-SNE), and pattern recognition (EM-GMM). **Change fps = camera frame-rate.**![Schematic](https://drive.google.com/uc?id=1_74Bdw8NThaMPj5r66uRMxMKpHz3aP2A)

In [ ]:
# change fps = camera frame-rate
f_10fps,tsne_feats,labels,tsne_fig = bsoid_assign(data,fps = 100,comp = 1,kclass = 50,it = 30) 

# Train a multiclass support vector machine (One vs. rest, kernel = Gaussian) classifier using pose as input, and clustered group as label. **Change hldout if you desire a different train/test ratio; change cv_it < 30 if you have a small dataset.**![Schematic](https://drive.google.com/uc?id=1AWcy4BOJ-h3obBEgEGjXyCkhI105MdTJ)

In [ ]:
# change hldout if you desire a different train/test ratio; change cv_it < 30 if you have a small dataset
clf,acc_fig,scores = bsoid_mdl2(f_10fps=f_10fps,labels=labels,hldout=0.2,cv_it=30) 

In [ ]:
# print(np.unique(labels),np.mean(scores))
# print(all_files)

# Plot multi-feature distributions (histograms by group).

In [ ]:
fig = plt.figure(num=None, figsize=(2, 3), dpi=300, facecolor='w', edgecolor='k')
R = np.linspace(0,1,len(np.unique(labels)))
color=plt.cm.hsv(R)
feat_ls = ("Distance between snout & center forepaw","Distance between snout & center hind paw","forepaw distance",
              "Body length","Angle","Snout speed","Proximal tail speed")
for j in range(0,f_10fps.shape[0]):
  fig = plt.figure(num=None, figsize=(2, 3), dpi=300, facecolor='w', edgecolor='k')
  for i in range(0,len(np.unique(labels))):
    plt.subplot(len(np.unique(labels)), 1, i+1)
    if j == 2 or j == 3 or j == 5 or j == 6:
      plt.hist(f_10fps[j,labels == i],
              bins = np.linspace(0,np.mean(f_10fps[j,:])+3*np.std(f_10fps[j,:])),
              range = (0,np.mean(f_10fps[j,:])+3*np.std(f_10fps[j,:])),
              color = color[i], density=True)
      plt.xticks(fontsize=6)
      plt.yticks(fontsize=6)
      fig.suptitle("{} pixels".format(feat_ls[j]), fontsize=8, fontweight='bold')
      plt.xlim(0,np.mean(f_10fps[j,:])+3*np.std(f_10fps[j,:]))
      if i < len(np.unique(labels))-1:
        plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    else:
      plt.hist(f_10fps[j,labels == i],
              bins = np.linspace(np.mean(f_10fps[j,:])-3*np.std(f_10fps[j,:]),np.mean(f_10fps[j,:])+3*np.std(f_10fps[j,:])),
              range = (np.mean(f_10fps[j,:])-3*np.std(f_10fps[j,:]),np.mean(f_10fps[j,:])+3*np.std(f_10fps[j,:])),
              color = color[i], density=True)
      plt.xticks(fontsize=6)
      plt.yticks(fontsize=6)
      plt.xlim(np.mean(f_10fps[j,:])-3*np.std(f_10fps[j,:]),np.mean(f_10fps[j,:])+3*np.std(f_10fps[j,:]))
      fig.suptitle("{} pixels".format(feat_ls[j]), fontsize=8, fontweight='bold')
      if i < len(np.unique(labels))-1:
        plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
plt.show()